In [1]:
import functools
import json
from pymongo import MongoClient

In [4]:
db=MongoClient()
stations = db.imfb
s_stations=[station for station in
                        sorted(list(stations.list_collections()), key=lambda station: station['name']) if
                        station['name'] not in ['logs','imdb', 'empties']]
s_stations

[{'name': 'imdb_export',
  'type': 'collection',
  'options': {},
  'info': {'readOnly': False,
   'uuid': UUID('2894d504-08f3-4aa3-b0e1-c616c6fc7683')},
  'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_'}}]

In [5]:
from openpyxl import Workbook
wb = Workbook(write_only=True)

In [6]:
def get_excel_row(row):
    if isinstance(row,list):
        output=[]
        for col in row:
            if isinstance(col,list) or isinstance(col,dict):
                output.append(json.dumps(col))
            else:
                output.append(col)
        return output
    elif isinstance(row,dict):
        output= {}
        for key,val in row.items():
            if isinstance(val,list) or  isinstance(val,dict) :
                output[key]=json.dumps(val)
            else:
                output[key]=val
        return output

In [7]:
for index, station in enumerate(s_stations):
    station_collection = stations[station['name']]
    ws=wb.create_sheet(station_collection.name)
    header={}
    for row in station_collection.find({}):
        header=get_excel_row(row)
        header.pop("_id")
        break
    ws.append(list(header.keys()))
    for row in station_collection.find({}):
        c_row=get_excel_row(row)
        c_row.pop("_id")
        ws.append(list(c_row.values()))
    print(f"Worked on {index} {station['name']}")

Worked on 0 imdb_export


In [6]:
wb.save(filename="2020_final_shows.xlsx")